Домашнее задание
Скачать из датасета Kinetics 700-2020 видео с классами содержащими слово dancing

Нельзя использовать веса предобученные на Kinetics!

Обучить модель на отдельных кадрах - 3 балла
Обучить модель классификации этих видео на основе Pose Estimation - 4 балла - взять модель для pose estimation, вытаскивать координаты точек
Построить с нуля и обучить модель классификации видео на основе 3D свёрток или трансформеров - 5 баллов - 3д свертки из пайторча, построить реснет с нуля

In [ ]:
!pip install datasets huggingface-hub transformers fiftyone decord av pytube -q

In [ ]:
from fiftyone.utils.kinetics import Kinetics7002020DatasetInfo
from fiftyone.utils.kinetics import KineticsDatasetDownloader
import tarfile
import os
import av
import numpy as np
from transformers import VivitImageProcessor, VivitModel
import numpy
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Prepare dataset

## Get dancing classes

In [2]:
kinetics_dir = "."

scratch_dir = '.'

split = "train"

dataset_info = Kinetics7002020DatasetInfo(kinetics_dir, scratch_dir, split)

Migrating database to v0.23.7


INFO:fiftyone.migrations.runner:Migrating database to v0.23.7


 100% |████|  189.7Mb/189.7Mb [368.3ms elapsed, 0s remaining, 515.1Mb/s]      


INFO:eta.core.utils: 100% |████|  189.7Mb/189.7Mb [368.3ms elapsed, 0s remaining, 515.1Mb/s]      


 100% |████|  388.3Kb/388.3Kb [68.0ms elapsed, 0s remaining, 6.5Mb/s] 


INFO:eta.core.utils: 100% |████|  388.3Kb/388.3Kb [68.0ms elapsed, 0s remaining, 6.5Mb/s] 


In [3]:
dancing_classes = [cls for cls in dataset_info.all_classes if "dancing" in cls]
dancing_classes

['belly dancing',
 'breakdancing',
 'country line dancing',
 'dancing ballet',
 'dancing charleston',
 'dancing gangnam style',
 'dancing macarena',
 'jumpstyle dancing',
 'mosh pit dancing',
 'robot dancing',
 'salsa dancing',
 'square dancing',
 'swing dancing',
 'tango dancing',
 'tap dancing']

## Download videos

In [8]:
downloader = KineticsDatasetDownloader(num_workers=None)

dataset_dir = "."

split = "train"

did_download = False
for cls in dancing_classes[:5]:
    try:
        downloader.download_classes(dataset_info, [cls])
        did_download = True
    except KeyError:
        print(f"Skipping class '{cls}' as it is not found in the dataset")


INFO:fiftyone.utils.kinetics:Downloading 1 tars...


 100% |██████|    5.6Gb/5.6Gb [23.6s elapsed, 0s remaining, 221.3Mb/s]      


INFO:eta.core.utils: 100% |██████|    5.6Gb/5.6Gb [23.6s elapsed, 0s remaining, 221.3Mb/s]      


INFO:fiftyone.utils.kinetics:Downloading 1 tars...


 100% |██████|    7.0Gb/7.0Gb [30.3s elapsed, 0s remaining, 274.6Mb/s]      


...

INFO:eta.core.utils: 100% |██████|    6.6Gb/6.6Gb [29.1s elapsed, 0s remaining, 249.0Mb/s]      


INFO:fiftyone.utils.kinetics:Downloading 1 tars...


 100% |██████|    8.2Gb/8.2Gb [35.8s elapsed, 0s remaining, 256.5Mb/s]      


INFO:eta.core.utils: 100% |██████|    8.2Gb/8.2Gb [35.8s elapsed, 0s remaining, 256.5Mb/s]      


Downloaded the dataset with the tag 'dancing'
Number of samples: None
Classes: ['belly dancing', 'breakdancing', 'country line dancing', 'dancing ballet', 'dancing charleston', 'dancing gangnam style', 'dancing macarena', 'jumpstyle dancing', 'mosh pit dancing', 'robot dancing', 'salsa dancing', 'square dancing', 'swing dancing', 'tango dancing', 'tap dancing']


## Unarchive and remove .tars

In [9]:
os.chdir("./")

for filename in os.listdir():
    if filename.startswith("k700") and filename.endswith(".tar.gz"):
        with tarfile.open(filename, "r:gz") as tar:
            tar.extractall("./kinetics")


In [11]:
subfolders = [f.path for f in os.scandir("./kinetics") if f.is_dir()]

for subfolder in subfolders:
    files = [f for f in os.listdir(subfolder)]

    files.sort()

    for file in files[5:]:
        file_path = os.path.join(subfolder, file)
        os.remove(file_path)



In [12]:
os.chdir("./")

files = os.listdir()

tar_gz_files = [f for f in files if f.endswith(".tar.gz")]

for f in tar_gz_files:
    os.remove(f)


# Train model

## Util functions

In [ ]:
np.random.seed(0)

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


def sample_frame_indices(clip_len, frame_sample_rate, seg_len):
    '''
    Sample a given number of frame indices from the video.
    Args:
        clip_len (`int`): Total number of frames to sample.
        frame_sample_rate (`int`): Sample every n-th frame.
        seg_len (`int`): Maximum allowed index of sample's last frame.
    Returns:
        indices (`List[int]`): List of sampled frame indices
    '''
    converted_len = int(clip_len * frame_sample_rate)
    end_idx = np.random.randint(converted_len, seg_len)
    start_idx = end_idx - converted_len
    indices = np.linspace(start_idx, end_idx, num=clip_len)
    indices = np.clip(indices, start_idx, end_idx - 1).astype(np.int64)
    return indices

## Dataset class

In [ ]:
dataset_dir = "./kinetics"

class KineticsDataset(torch.utils.data.Dataset):
    def __init__(self, dataset_dir, transform=None):
        self.dataset_dir = dataset_dir
        self.classes = sorted(os.listdir(dataset_dir))
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}
        self.samples = []
        for cls in self.classes:
            class_dir = os.path.join(dataset_dir, cls)
            for video_file in os.listdir(class_dir):
                self.samples.append((os.path.join(class_dir, video_file), self.class_to_idx[cls]))
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        video_path, label = self.samples[idx]
        return video_path, label


## Init

In [ ]:
image_processor = VivitImageProcessor.from_pretrained("google/vivit-b-16x2")
model = VivitModel.from_pretrained("google/vivit-b-16x2")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_epochs = 10
batch_size = 1
learning_rate = 1e-4

train_dataset = KineticsDataset(dataset_dir)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

## Training loop

In [15]:
for epoch in range(num_epochs):
    total_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    for video_paths, labels in tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=False):
        videos = []
        inputs = []
        for video_path in video_paths:
            container = av.open(video_path)
            indices = sample_frame_indices(clip_len=32, frame_sample_rate=1, seg_len=container.streams.video[0].frames)
            video_frames = read_video_pyav(container, indices)
            videos.append(video_frames)
            processed_frames = image_processor(list(numpy.array(video_frames)), return_tensors="pt")
            inputs.append(processed_frames['pixel_values'])
            container.close()

        outputs = model(inputs[0].to(device))

        loss = loss_fn(outputs.pooler_output, labels.to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item() * len(video_paths)
        _, predicted = torch.max(outputs.pooler_output, 1)
        correct_predictions += (predicted == labels.to(device)).sum().item()
        total_samples += len(video_paths)

    epoch_loss = total_loss / total_samples
    accuracy = correct_predictions / total_samples

    print(f"Epoch {epoch+1}/{num_epochs}: Loss: {epoch_loss}, Accuracy: {accuracy}")


Epoch 1/10:   0%|          | 0/25 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/feature_extraction_utils.py:141: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  return torch.tensor(value)


Epoch 1/10: Loss: 6.045347785949707, Accuracy: 0.12


Epoch 2/10: Loss: 5.006286067962646, Accuracy: 0.4


Epoch 3/10: Loss: 4.754126701354981, Accuracy: 0.36


Epoch 4/10: Loss: 4.718386764526367, Accuracy: 0.4


...

Epoch 6/10: Loss: 4.702522392272949, Accuracy: 0.56


Epoch 7/10: Loss: 4.698419742584228, Accuracy: 0.56


Epoch 8/10: Loss: 4.695784111022949, Accuracy: 0.52


Epoch 9/10: Loss: 4.693986167907715, Accuracy: 0.52


Epoch 10/10: Loss: 4.692481307983399, Accuracy: 0.6
